# Self Driving Car - CNN

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Lambda
from keras.layers import MaxPooling2D, Dropout,BatchNormalization
from keras.utils import print_summary
import tensorflow as tf
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import pickle
from keras.optimizers import Adam
import scipy.misc
import cv2
from subprocess import call
import math
from keras.callbacks import EarlyStopping
from tqdm import tqdm_notebook as tqdm
from itertools import islice
from keras.models import load_model
from ann_visualizer.visualize import ann_viz
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [5]:
# utility function to load data
def load_data(start_pointer,end_pointer):
    print('Loading Data Start...')
    images=[]
    labels=[]
    xs=[]
    ys=[]
    with open("driving_dataset/data.txt") as f:
        for line in f:
            ys.append(float(line.split()[1]) * scipy.pi / 180)
            labels.append("driving_dataset/" + line.split()[0])
        print('No of image files:',len(ys))
        print('No of labels appended:',len(labels))
        ys=ys[start_pointer:end_pointer]
        labels=labels[start_pointer:end_pointer]
        print('Starting index of current batch:',start_pointer)
        print('End index of current batch:',end_pointer)
        print('No of image files for current batch:',len(ys))
        for file in tqdm(labels):
            image = cv2.imread(file) 
            image = cv2.resize(image,(200,66))
            images.append(image)
        images=np.array(images).astype('float32')
        ys=np.array(ys).astype('float32')
        print('Data loaded sucessfully.')
        return images,ys
    

In [9]:
# configurations
batch_size=100
epochs=30
dropout_rate=0.5

In [10]:
# CNN architecture
input_shape=(66,200,3)
def run_model(lr_rate):
    model=Sequential()
    # conv layer1
    model.add(Conv2D(32,kernel_size=(5, 5),activation='relu',input_shape=input_shape,padding='same'))
    model.add(MaxPooling2D((2, 2), padding='valid'))
    #conv layer2
    model.add(Conv2D(64,kernel_size=(5,5),activation='relu'))
    model.add(MaxPooling2D((2, 2), padding='valid'))
    model.add(Dropout(0.4))
    #conv layer3
    model.add(BatchNormalization())
    model.add(Conv2D(128,kernel_size=(5,5),activation='relu'))
    model.add(Dropout(0.4))
    #conv layer4
    model.add(BatchNormalization())
    model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
    model.add(Dropout(0.4))
    #conv layer5
    model.add(BatchNormalization())
    model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
    model.add(MaxPooling2D((2, 2), padding='valid'))
    model.add(Dropout(0.4))
    #Flatten
    model.add(Flatten())
    model.add(Dropout(0.4))
    # Densly connected layers
    model.add(Dense(128))
    model.add(Dense(64))
    # output layer
    model.add(Dense(1))
    model.compile(optimizer=Adam(lr=lr_rate), loss="mse")
    filepath = "DriverModel.h5"
    es=EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    #checkpoint1 = ModelCheckpoint(filepath, verbose=1, save_best_only=True)
    callbacks_list = [es]
    print(model.summary())
    return model, callbacks_list
  
      

In [11]:
def get_data_iterator(x,y,batch_size=batch_size):
    train_x, test_x, train_y, test_y = train_test_split(x, y, random_state=0,
                                                        test_size=0.2)
    train_datagen=ImageDataGenerator(featurewise_std_normalization=True)
    train_datagen.fit(train_x)
    validation_datagen=ImageDataGenerator(featurewise_std_normalization=True)
    validation_datagen.fit(test_x)
    train_itr=train_datagen.flow(train_x,train_y,batch_size=batch_size)
    validation_itr=validation_datagen.flow(test_x,test_y,batch_size=batch_size)
    return train_itr,validation_itr

In [31]:
model,callbacks=run_model(lr_rate=0.0001)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 66, 200, 32)       2432      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 33, 100, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 96, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 48, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 48, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 48, 64)        256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 44, 128)       204928    
__________

### Loading data from 0-10000:

In [37]:
x,y=load_data(0,10000)

Loading Data Start...
No of image files: 45406
No of labels appended: 45406
Starting index of current batch: 0
End index of current batch: 10000
No of image files for current batch: 10000



Data loaded sucessfully.


In [38]:
train_x, test_x, train_y, test_y = train_test_split(x, y, random_state=0,
                                                        test_size=0.2)

#### Epoch 1 to 5:

In [40]:
es=EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.fit(train_x,train_y,validation_data=[test_x,test_y],epochs=5,batch_size=batch_size,callbacks=[es])

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - ETA: 16:24 - loss: 15.611 - ETA: 14:20 - loss: 18.876 - ETA: 13:32 - loss: 25.188 - ETA: 13:00 - loss: 23.928 - ETA: 12:39 - loss: 22.915 - ETA: 12:22 - loss: 23.008 - ETA: 12:08 - loss: 21.213 - ETA: 11:53 - loss: 20.407 - ETA: 11:41 - loss: 21.076 - ETA: 11:28 - loss: 20.682 - ETA: 11:16 - loss: 20.293 - ETA: 11:05 - loss: 20.050 - ETA: 10:54 - loss: 19.534 - ETA: 10:43 - loss: 19.148 - ETA: 10:33 - loss: 19.011 - ETA: 10:22 - loss: 18.755 - ETA: 10:12 - loss: 18.451 - ETA: 10:01 - loss: 18.271 - ETA: 9:51 - loss: 18.024 - ETA: 9:41 - loss: 17.74 - ETA: 9:31 - loss: 17.44 - ETA: 9:20 - loss: 17.51 - ETA: 9:10 - loss: 17.33 - ETA: 9:00 - loss: 17.06 - ETA: 8:50 - loss: 17.09 - ETA: 8:40 - loss: 16.99 - ETA: 8:30 - loss: 16.83 - ETA: 8:21 - loss: 16.77 - ETA: 8:11 - loss: 16.66 - ETA: 8:01 - loss: 16.55 - ETA: 7:52 - loss: 16.42 - ETA: 7:42 - loss: 16.34 - ETA: 7:33 - loss: 16.23 - ET

#### Epoch 6 to 16:

In [46]:
es=EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.fit(train_x,train_y,validation_data=[test_x,test_y],epochs=10,batch_size=batch_size,callbacks=[es])

Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - ETA: 7:07 - loss: 1.312 - ETA: 6:57 - loss: 1.207 - ETA: 6:58 - loss: 1.139 - ETA: 7:00 - loss: 1.117 - ETA: 7:02 - loss: 1.044 - ETA: 7:02 - loss: 1.132 - ETA: 7:02 - loss: 1.143 - ETA: 7:00 - loss: 1.130 - ETA: 7:02 - loss: 1.080 - ETA: 6:59 - loss: 1.073 - ETA: 7:01 - loss: 1.076 - ETA: 7:08 - loss: 1.049 - ETA: 7:14 - loss: 1.039 - ETA: 7:23 - loss: 1.051 - ETA: 7:28 - loss: 1.045 - ETA: 7:26 - loss: 1.064 - ETA: 7:23 - loss: 1.134 - ETA: 7:16 - loss: 1.117 - ETA: 7:08 - loss: 1.121 - ETA: 7:00 - loss: 1.113 - ETA: 6:53 - loss: 1.158 - ETA: 6:45 - loss: 1.162 - ETA: 6:38 - loss: 1.174 - ETA: 6:31 - loss: 1.188 - ETA: 6:23 - loss: 1.175 - ETA: 6:16 - loss: 1.179 - ETA: 6:09 - loss: 1.178 - ETA: 6:02 - loss: 1.156 - ETA: 5:54 - loss: 1.155 - ETA: 5:47 - loss: 1.145 - ETA: 5:40 - loss: 1.147 - ETA: 5:33 - loss: 1.147 - ETA: 5:26 - loss: 1.138 - ETA: 5:19 - loss: 1.151 - ETA: 5:12 - 

8000/8000 [==============================] - ETA: 9:00 - loss: 1.087 - ETA: 8:57 - loss: 1.004 - ETA: 8:53 - loss: 0.872 - ETA: 8:46 - loss: 0.908 - ETA: 8:39 - loss: 0.827 - ETA: 8:32 - loss: 0.783 - ETA: 8:25 - loss: 0.717 - ETA: 8:18 - loss: 0.703 - ETA: 8:12 - loss: 0.681 - ETA: 8:05 - loss: 0.647 - ETA: 7:58 - loss: 0.619 - ETA: 7:51 - loss: 0.612 - ETA: 7:45 - loss: 0.586 - ETA: 7:38 - loss: 0.577 - ETA: 7:31 - loss: 0.558 - ETA: 7:24 - loss: 0.561 - ETA: 7:17 - loss: 0.558 - ETA: 7:10 - loss: 0.558 - ETA: 7:03 - loss: 0.542 - ETA: 6:56 - loss: 0.555 - ETA: 6:49 - loss: 0.603 - ETA: 6:43 - loss: 0.594 - ETA: 6:36 - loss: 0.604 - ETA: 6:29 - loss: 0.590 - ETA: 6:22 - loss: 0.583 - ETA: 6:15 - loss: 0.576 - ETA: 6:08 - loss: 0.583 - ETA: 6:01 - loss: 0.583 - ETA: 5:54 - loss: 0.579 - ETA: 5:49 - loss: 0.582 - ETA: 5:42 - loss: 0.582 - ETA: 5:35 - loss: 0.602 - ETA: 5:28 - loss: 0.606 - ETA: 5:22 - loss: 0.600 - ETA: 5:15 - loss: 0.597 - ETA: 5:07 - loss: 0.591 - ETA: 5:00 - loss: 0

In [49]:
model.save('model_59.h5')

In [14]:
model=load_model('model_59.h5')

Instructions for updating:
Use tf.cast instead.


### Loading Data from 10,000 to 20,000

In [15]:
x,y=load_data(10000,20000)

Loading Data Start...
No of image files: 45406
No of labels appended: 45406
Starting index of current batch: 10000
End index of current batch: 20000
No of image files for current batch: 10000



Data loaded sucessfully.


In [16]:
train_x, test_x, train_y, test_y = train_test_split(x, y, random_state=0,
                                                        test_size=0.2)

In [17]:
model.evaluate(test_x,test_y)

2000/2000 [==============================] - ETA: 4: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 58s - ETA: 55 - ETA: 53 - ETA: 51 - ETA: 49 - ETA: 47 - ETA: 46 - ETA: 44 - ETA: 43 - ETA: 41 - ETA: 40 - ETA: 39 - ETA: 38 - ETA: 36 - ETA: 35 - ETA: 34 - ETA: 33 - ETA: 32 - ETA: 31 - ETA: 30 - ETA: 29 - ETA: 28 - ETA: 27 - ETA: 26 - ETA: 25 - ETA: 24 - ETA: 23 - ETA: 22 - ETA: 21 - ETA: 20 - ETA: 19 - ETA: 18 - ETA: 17 - ETA: 16 - ETA: 15 - ETA: 14 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 56s 28ms/step


0.2030391857624054

#### Epoch 1 to 10 with batch2(10,000 to 20,000 data points):

In [60]:
model.fit_generator(train_datagen.flow(train_x,train_y,steps_per_epoch=len(train_x)/128,epochs=10)

Epoch 1/10
63/62 [==============================] - ETA: 9:45 - loss: 0.277 - ETA: 9:35 - loss: 0.401 - ETA: 9:27 - loss: 0.334 - ETA: 9:18 - loss: 0.305 - ETA: 9:08 - loss: 0.273 - ETA: 9:00 - loss: 0.254 - ETA: 8:50 - loss: 0.258 - ETA: 8:40 - loss: 0.263 - ETA: 8:30 - loss: 0.264 - ETA: 8:19 - loss: 0.259 - ETA: 8:08 - loss: 0.254 - ETA: 7:59 - loss: 0.261 - ETA: 7:50 - loss: 0.265 - ETA: 7:40 - loss: 0.262 - ETA: 7:31 - loss: 0.261 - ETA: 7:22 - loss: 0.254 - ETA: 7:08 - loss: 0.251 - ETA: 6:50 - loss: 0.247 - ETA: 6:34 - loss: 0.248 - ETA: 6:19 - loss: 0.247 - ETA: 6:07 - loss: 0.243 - ETA: 5:53 - loss: 0.244 - ETA: 5:40 - loss: 0.240 - ETA: 5:28 - loss: 0.239 - ETA: 5:16 - loss: 0.235 - ETA: 5:04 - loss: 0.235 - ETA: 4:53 - loss: 0.237 - ETA: 4:42 - loss: 0.236 - ETA: 4:31 - loss: 0.233 - ETA: 4:21 - loss: 0.230 - ETA: 4:11 - loss: 0.230 - ETA: 4:01 - loss: 0.228 - ETA: 3:52 - loss: 0.224 - ETA: 3:42 - loss: 0.223 - ETA: 3:33 - loss: 0.222 - ETA: 3:24 - loss: 0.220 - ETA: 3:16 - 

63/62 [==============================] - ETA: 6:39 - loss: 0.191 - ETA: 6:34 - loss: 0.141 - ETA: 6:29 - loss: 0.137 - ETA: 6:24 - loss: 0.127 - ETA: 6:19 - loss: 0.115 - ETA: 6:15 - loss: 0.104 - ETA: 6:09 - loss: 0.120 - ETA: 6:04 - loss: 0.119 - ETA: 5:58 - loss: 0.112 - ETA: 5:52 - loss: 0.111 - ETA: 5:46 - loss: 0.116 - ETA: 5:39 - loss: 0.114 - ETA: 5:33 - loss: 0.108 - ETA: 5:26 - loss: 0.105 - ETA: 5:19 - loss: 0.109 - ETA: 5:13 - loss: 0.111 - ETA: 5:07 - loss: 0.109 - ETA: 5:00 - loss: 0.110 - ETA: 4:53 - loss: 0.109 - ETA: 4:47 - loss: 0.113 - ETA: 4:40 - loss: 0.111 - ETA: 4:33 - loss: 0.111 - ETA: 4:27 - loss: 0.111 - ETA: 4:20 - loss: 0.111 - ETA: 4:13 - loss: 0.113 - ETA: 4:07 - loss: 0.113 - ETA: 4:00 - loss: 0.114 - ETA: 3:53 - loss: 0.115 - ETA: 3:46 - loss: 0.114 - ETA: 3:40 - loss: 0.114 - ETA: 3:33 - loss: 0.113 - ETA: 3:26 - loss: 0.114 - ETA: 3:20 - loss: 0.112 - ETA: 3:13 - loss: 0.112 - ETA: 3:06 - loss: 0.113 - ETA: 3:00 - loss: 0.112 - ETA: 2:53 - loss: 0.111

In [26]:
es=EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.fit(train_x,train_y,validation_data=[test_x,test_y],epochs=5,batch_size=batch_size,callbacks=[es])

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - ETA: 13:40 - loss: 0.08 - ETA: 12:04 - loss: 0.17 - ETA: 11:30 - loss: 0.16 - ETA: 12:01 - loss: 0.18 - ETA: 11:37 - loss: 0.19 - ETA: 11:11 - loss: 0.17 - ETA: 11:46 - loss: 0.16 - ETA: 11:41 - loss: 0.15 - ETA: 11:27 - loss: 0.15 - ETA: 12:07 - loss: 0.15 - ETA: 11:48 - loss: 0.16 - ETA: 11:42 - loss: 0.15 - ETA: 11:26 - loss: 0.15 - ETA: 11:13 - loss: 0.16 - ETA: 11:01 - loss: 0.16 - ETA: 10:47 - loss: 0.15 - ETA: 10:33 - loss: 0.15 - ETA: 10:24 - loss: 0.15 - ETA: 10:10 - loss: 0.15 - ETA: 9:55 - loss: 0.1520 - ETA: 9:53 - loss: 0.152 - ETA: 9:44 - loss: 0.151 - ETA: 9:35 - loss: 0.148 - ETA: 9:28 - loss: 0.149 - ETA: 9:19 - loss: 0.154 - ETA: 9:09 - loss: 0.152 - ETA: 8:57 - loss: 0.154 - ETA: 8:45 - loss: 0.153 - ETA: 8:33 - loss: 0.155 - ETA: 8:26 - loss: 0.154 - ETA: 8:18 - loss: 0.156 - ETA: 8:03 - loss: 0.154 - ETA: 7:53 - loss: 0.154 - ETA: 7:41 - loss: 0.151 - ETA: 7:29 - 

### Loading Data From 20,000 to 35,000:

In [32]:
x,y=load_data(20000,35000)

Loading Data Start...
No of image files: 45406
No of labels appended: 45406
Starting index of current batch: 20000
End index of current batch: 35000
No of image files for current batch: 15000


Data loaded sucessfully.


In [33]:
train_x, test_x, train_y, test_y = train_test_split(x, y, random_state=0,
                                                        test_size=0.2)

In [39]:
model.evaluate(test_x,test_y)

3000/3000 [==============================] - ETA: 3: - ETA: 2: - ETA: 3: - ETA: 3: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 59s - ETA: 57 - ETA: 55 - ETA: 52 - ETA: 50 - ETA: 48 - ETA: 46 - ETA: 44 - ETA: 42 - ETA: 40 - ETA: 38 - ETA: 37 - ETA: 35 - ETA: 33 - ETA: 31 - ETA: 29 - ETA: 27 - ETA: 25 - ETA: 23 - ETA: 22 - ETA: 20 - ETA: 18 - ETA: 17 - ETA: 15 - ETA: 14 - ETA: 12 - ETA: 11 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 152s 51ms/step


0.23768171318372092

#### Epoch 1 to 5 with batch3(20,000 to 35,000 datapoints)

In [40]:
es=EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.fit(train_x,train_y,validation_data=[test_x,test_y],epochs=5,batch_size=200,callbacks=[es])

Train on 12000 samples, validate on 3000 samples
Epoch 1/5
12000/12000 [==============================] - ETA: 17:21 - loss: 0.21 - ETA: 17:03 - loss: 0.23 - ETA: 16:39 - loss: 0.22 - ETA: 16:23 - loss: 0.22 - ETA: 16:11 - loss: 0.23 - ETA: 16:20 - loss: 0.24 - ETA: 16:02 - loss: 0.24 - ETA: 15:52 - loss: 0.25 - ETA: 15:30 - loss: 0.25 - ETA: 15:10 - loss: 0.25 - ETA: 14:53 - loss: 0.25 - ETA: 14:45 - loss: 0.25 - ETA: 14:28 - loss: 0.24 - ETA: 14:09 - loss: 0.24 - ETA: 13:58 - loss: 0.24 - ETA: 13:37 - loss: 0.24 - ETA: 13:17 - loss: 0.24 - ETA: 13:04 - loss: 0.24 - ETA: 12:42 - loss: 0.24 - ETA: 12:32 - loss: 0.24 - ETA: 12:15 - loss: 0.24 - ETA: 11:55 - loss: 0.24 - ETA: 11:36 - loss: 0.24 - ETA: 11:16 - loss: 0.24 - ETA: 10:57 - loss: 0.24 - ETA: 10:37 - loss: 0.24 - ETA: 10:19 - loss: 0.24 - ETA: 10:01 - loss: 0.24 - ETA: 9:42 - loss: 0.2484 - ETA: 9:24 - loss: 0.247 - ETA: 9:05 - loss: 0.248 - ETA: 8:47 - loss: 0.248 - ETA: 8:27 - loss: 0.246 - ETA: 8:09 - loss: 0.245 - ETA: 7:49

In [41]:
model.evaluate(test_x,test_y)

3000/3000 [==============================] - ETA: 57 - ETA: 52 - ETA: 49 - ETA: 47 - ETA: 45 - ETA: 45 - ETA: 44 - ETA: 44 - ETA: 43 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 42 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 41 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 40 - ETA: 39 - ETA: 39 - ETA: 38 - ETA: 38 - ETA: 37 - ETA: 37 - ETA: 36 - ETA: 36 - ETA: 35 - ETA: 34 - ETA: 34 - ETA: 33 - ETA: 32 - ETA: 32 - ETA: 31 - ETA: 31 - ETA: 30 - ETA: 29 - ETA: 29 - ETA: 28 - ETA: 27 - ETA: 27 - ETA: 26 - ETA: 25 - ETA: 25 - ETA: 24 - ETA: 23 - ETA: 23 - ETA: 22 - ETA: 21 - ETA: 21 - ETA: 20 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 63s 21ms/step


0.1727147475083669

In [44]:
model.save('model_0.17_loss.h5')

### Loading Data From 35,000 to 45,000:

In [45]:
x,y=load_data(35000,45000)
train_x, test_x, train_y, test_y = train_test_split(x, y, random_state=0,
                                                        test_size=0.2)

Loading Data Start...
No of image files: 45406
No of labels appended: 45406
Starting index of current batch: 35000
End index of current batch: 45000
No of image files for current batch: 10000


Data loaded sucessfully.


#### Epoch 1 to 5 with batch3(35,000 to 45,000 datapoints)

In [48]:
es=EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.fit(train_x,train_y,validation_data=[test_x,test_y],epochs=5,batch_size=100,callbacks=[es])

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - ETA: 14:59 - loss: 0.52 - ETA: 13:11 - loss: 0.39 - ETA: 12:58 - loss: 0.32 - ETA: 12:19 - loss: 0.26 - ETA: 12:00 - loss: 0.23 - ETA: 11:42 - loss: 0.22 - ETA: 11:21 - loss: 0.22 - ETA: 11:04 - loss: 0.22 - ETA: 10:48 - loss: 0.22 - ETA: 10:35 - loss: 0.21 - ETA: 10:21 - loss: 0.21 - ETA: 10:07 - loss: 0.21 - ETA: 9:59 - loss: 0.2141 - ETA: 9:48 - loss: 0.224 - ETA: 9:39 - loss: 0.224 - ETA: 9:27 - loss: 0.230 - ETA: 9:18 - loss: 0.229 - ETA: 9:12 - loss: 0.237 - ETA: 9:03 - loss: 0.236 - ETA: 8:55 - loss: 0.234 - ETA: 8:48 - loss: 0.233 - ETA: 8:40 - loss: 0.229 - ETA: 8:34 - loss: 0.225 - ETA: 8:29 - loss: 0.223 - ETA: 8:20 - loss: 0.218 - ETA: 8:10 - loss: 0.220 - ETA: 8:02 - loss: 0.220 - ETA: 7:53 - loss: 0.228 - ETA: 7:43 - loss: 0.224 - ETA: 7:33 - loss: 0.223 - ETA: 7:23 - loss: 0.223 - ETA: 7:13 - loss: 0.223 - ETA: 7:04 - loss: 0.229 - ETA: 6:55 - loss: 0.225 - ETA: 6:45 - 

#### Epoch 6 to 10:

In [60]:
es=EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.fit(train_x,train_y,validation_data=[test_x,test_y],epochs=5,batch_size=100,callbacks=[es])

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - ETA: 12:46 - loss: 0.22 - ETA: 11:44 - loss: 0.18 - ETA: 11:32 - loss: 0.16 - ETA: 11:34 - loss: 0.15 - ETA: 11:15 - loss: 0.14 - ETA: 10:58 - loss: 0.15 - ETA: 10:43 - loss: 0.15 - ETA: 10:30 - loss: 0.16 - ETA: 10:18 - loss: 0.17 - ETA: 10:08 - loss: 0.17 - ETA: 9:58 - loss: 0.1729 - ETA: 9:47 - loss: 0.166 - ETA: 9:37 - loss: 0.163 - ETA: 9:27 - loss: 0.159 - ETA: 9:18 - loss: 0.159 - ETA: 9:08 - loss: 0.158 - ETA: 8:59 - loss: 0.155 - ETA: 8:50 - loss: 0.153 - ETA: 8:40 - loss: 0.160 - ETA: 8:32 - loss: 0.162 - ETA: 8:25 - loss: 0.165 - ETA: 8:15 - loss: 0.165 - ETA: 8:07 - loss: 0.166 - ETA: 7:58 - loss: 0.164 - ETA: 7:49 - loss: 0.162 - ETA: 7:40 - loss: 0.158 - ETA: 7:33 - loss: 0.156 - ETA: 7:24 - loss: 0.155 - ETA: 7:16 - loss: 0.154 - ETA: 7:08 - loss: 0.154 - ETA: 6:59 - loss: 0.155 - ETA: 6:51 - loss: 0.157 - ETA: 6:43 - loss: 0.158 - ETA: 6:35 - loss: 0.160 - ETA: 6:27 - 

#### Evaluate loss:

In [15]:
# Evaluation loss
model.evaluate(test_x,test_y)

2000/2000 [==============================] - ETA: 3: - ETA: 2: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 56s - ETA: 52 - ETA: 49 - ETA: 46 - ETA: 44 - ETA: 42 - ETA: 40 - ETA: 39 - ETA: 37 - ETA: 36 - ETA: 35 - ETA: 34 - ETA: 32 - ETA: 31 - ETA: 30 - ETA: 30 - ETA: 29 - ETA: 28 - ETA: 27 - ETA: 26 - ETA: 25 - ETA: 24 - ETA: 23 - ETA: 23 - ETA: 22 - ETA: 21 - ETA: 20 - ETA: 20 - ETA: 19 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 43s 21ms/step


0.14316689416766168

#### Evaluate Predicted Values:

In [72]:
# checking predicted values
for i in range(20):
    print('Predicted:',y_pred[i][0],'Actual:',test_y[i])

Predicted: 0.14099488 Actual: 0.1706932
Predicted: 0.03689584 Actual: -0.1425934
Predicted: 0.11584562 Actual: 0.31503192
Predicted: -0.030835485 Actual: -0.010646508
Predicted: -0.0034901632 Actual: 0.38362336
Predicted: -0.17210875 Actual: -0.087964594
Predicted: 0.27801698 Actual: 0.03874631
Predicted: 0.17287785 Actual: 0.07574729
Predicted: 0.32338747 Actual: -0.0052359877
Predicted: 0.06406954 Actual: 0.21816616
Predicted: -0.16901803 Actual: -0.07749262
Predicted: 0.05489731 Actual: 0.20769419
Predicted: 0.009950997 Actual: 0.10035643
Predicted: -0.13463046 Actual: -0.03700098
Predicted: 0.14824441 Actual: 0.14433873
Predicted: 0.08418703 Actual: 0.0
Predicted: -0.15607047 Actual: -0.006981317
Predicted: 0.15003757 Actual: 0.14433873
Predicted: 0.2689249 Actual: -0.028099801
Predicted: -0.017937494 Actual: 0.03176499


In [24]:
# utility to visualize model performance
import tensorflow as tf
import scipy.misc
import cv2
from subprocess import call
import math
import time
print('Recording will start after 10s..')
time.sleep(10)
print('Recording started..')
img = cv2.imread('steering_wheel_image.jpg',0)
rows,cols = img.shape
smoothed_angle = 0
xs = []
ys = []
with open("driving_dataset/data.txt") as f:
    for line in f:
        xs.append("driving_dataset/" + line.split()[0])
        ys.append(float(line.split()[1]) * scipy.pi / 180)

num_images = len(xs)
i = math.ceil(num_images*0.8)
for file in xs:
    full_image = cv2.imread(file) 
    image = cv2.resize(full_image,(200,66))
    image=np.reshape(image, (-1, 66, 200, 3))
    degrees = float(model.predict(image,batch_size=1)) * 180.0 / scipy.pi
    cv2.imshow("frame", cv2.cvtColor(full_image, cv2.COLOR_RGB2BGR))
    smoothed_angle += 0.2 * pow(abs((degrees - smoothed_angle)), 2.0 / 3.0) * (degrees - smoothed_angle) / abs(degrees - smoothed_angle)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),-smoothed_angle,1)
    dst = cv2.warpAffine(img,M,(cols,rows))
    cv2.imshow("steering wheel", dst)
    i += 1
    cv2.waitKey(10)
cv2.destroyAllWindows()
